In [4]:
import networkx as nx
import random

# Generate a random graph
graph = nx.erdos_renyi_graph(100, 0.05)

# Set node thresholds and edge weights
for node in graph.nodes():
    graph.nodes[node]["threshold"] = random.randint(1, 10)
for edge in graph.edges():
    graph[edge[0]][edge[1]]["weight"] = random.uniform(0.1, 1.0)

In [5]:
def ic_model(graph, seeds):
    active_nodes = set(seeds)
    state = {node: "inactive" for node in graph.nodes()}
    for seed in seeds:
        state[seed] = "active"
    queue = list(seeds)
    while len(queue) > 0:
        node = queue.pop()
        for neighbor in graph[node]:
            if state[neighbor] == "inactive":
                weight = graph[node][neighbor]["weight"]
                threshold = graph.nodes[neighbor]["threshold"]
                threshold_sum = sum([graph[neighbor][v]["weight"] for v in graph[neighbor] if state[v] == "active"])
                if threshold_sum + weight >= threshold:
                    state[neighbor] = "active"
                    active_nodes.add(neighbor)
                    queue.append(neighbor)
                else:
                    state[neighbor] = "expired"
    return len(active_nodes)

# Test the ic_model function
seeds = [1, 2, 3]
max_influence = ic_model(graph, seeds)
print("Max influence:", max_influence)

Max influence: 3


In [6]:
def ic_model_with_pruning(graph, seeds):
    active_nodes = set(seeds)
    state = {node: "inactive" for node in graph.nodes()}
    for seed in seeds:
        state[seed] = "active"
    queue = list(seeds)
    while len(queue) > 0:
        node = queue.pop()
        for neighbor in graph[node]:
            if state[neighbor] == "inactive":
                weight = graph[node][neighbor]["weight"]
                threshold = graph.nodes[neighbor]["threshold"]
                threshold_sum = sum([graph[neighbor][v]["weight"] for v in graph[neighbor] if state[v] == "active"])
                if threshold_sum + weight >= threshold:
                    state[neighbor] = "active"
                    active_nodes.add(neighbor)
                    queue.append(neighbor)
                else:
                    state[neighbor] = "expired"
            elif state[neighbor] == "active":
                weight = graph[node][neighbor]["weight"]
                threshold = graph.nodes[neighbor]["threshold"]
                threshold_sum = sum([graph[neighbor][v]["weight"] for v in graph[neighbor] if state[v] == "active"])
                if threshold_sum + weight < threshold:
                    state[neighbor] = "expired"
    return len(active_nodes)

# Test the ic_model_with_pruning function
seeds = [1, 2, 3]
max_influence = ic_model_with_pruning(graph, seeds)
print("Max influence with pruning:", max_influence)

Max influence with pruning: 3


In [7]:
def greedy_algorithm(graph, k):
    seeds = set()
    influence = ic_model_with_pruning(graph, seeds)
    while len(seeds) < k:
        max_node = None
        max_increase = -1
        for node in graph.nodes():
            if node not in seeds:
                new_seeds = seeds | {node}
                new_influence = ic_model_with_pruning(graph, new_seeds)
                increase = new_influence - influence
                if increase > max_increase:
                    max_node = node
                    max_increase = increase
        if max_increase > 0:
            seeds.add(max_node)
            influence += max_increase
        else:
            break
    return seeds, influence

# Test the greedy_algorithm function
seeds, max_influence = greedy_algorithm(graph, 10)
print("Greedy algorithm result:")
print("Seeds:", seeds)
print("Max influence:", max_influence)

Greedy algorithm result:
Seeds: {0, 4, 37, 71, 8, 44, 50, 26}
Max influence: 23


In [8]:
#最后，我们可以使用基于深度学习的方法来选择种子节点，实现代码如下所示：

import torch
import torch.nn as nn
import torch.optim as optim

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_net(graph, features, labels):
    net = Net(len(features[0]), 64, 1)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.01)
    # 将features从字典转换为张量
    feature_tensor = torch.tensor([features[node] for node in graph.nodes()], dtype=torch.float32)
    labels = labels.clone().detach().unsqueeze(1)

    for epoch in range(100):
        optimizer.zero_grad()
        outputs = net(feature_tensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return net

def predict_influence(net, features, nodes):
    inputs = []
    for node in nodes:
        inputs.append(features[node])
    # inputs = torch.tensor([features[node] for node in graph.nodes()], dtype=torch.float32)
    inputs = torch.tensor(inputs, dtype=torch.float32)
    with torch.no_grad():
        outputs = net(inputs)
        influence = {}
        for node in nodes:
            influence[node] = outputs[node].item()
        # for i in range(len(nodes)):
        #     influence[nodes[i]["threshold"]] = outputs[i].item()
    return influence

def select_seeds(influence, k):
    seeds = []
    sorted_influence = sorted(influence.items(), key=lambda x: x[1], reverse=True)
    for i in range(k):
        seeds.append(sorted_influence[i][0])
    return seeds

def deep_learning_algorithm(graph, features, k):
    labels = []
    for node in graph.nodes():
        labels.append(ic_model_with_pruning(graph, [node]))
    labels = torch.tensor(labels, dtype=torch.float32)
    net = train_net(graph, features, labels)
    influence = predict_influence(net, features, graph.nodes())

    seeds = select_seeds(influence, k)
    max_influence = ic_model_with_pruning(graph, seeds)
    return seeds, max_influence

# Test the deep_learning_algorithm function
features = {node: [random.uniform(0.0, 1.0) for _ in range(10)] for node in graph.nodes()}
print(features[0])
seeds, max_influence = deep_learning_algorithm(graph, features, 10)
print("Deep learning algorithm result:")
print("Seeds:", seeds)
print("Max influence:", max_influence)
# 以上就是完整的影响力最大化例子，其中包含了基于独立级联模型、剪枝策略、贪心算法和基于深度学习的方法。需要注意的是，该例子中使用的影响力传播模型和节点特征提取方法仅供参考，在实际应用中需要根据具体问题进行选择和调整。

[0.30538110193613344, 0.2181748328136004, 0.5858577872628989, 0.2849953143526075, 0.5991080145150262, 0.9909784666250216, 0.806740405600709, 0.008713328933675601, 0.7878332317852551, 0.8579186826564871]
Deep learning algorithm result:
Seeds: [84, 97, 32, 39, 33, 71, 67, 80, 89, 78]
Max influence: 18
